In [7]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.2/spark-3.4.2-bin-hadoop3.tgz
!tar xf spark-3.4.2-bin-hadoop3.tgz
!pip install -q findspark

In [30]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.2-bin-hadoop3"
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, col
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [10]:
spark = SparkSession.builder.master('local').appName('Covid Data Analysis').getOrCreate()

# Выберите 15 стран с наибольшим процентом переболевших на 31 марта (в выходящем датасете необходимы колонки: iso_code, страна, процент переболевших)

In [20]:
# Определение максимальной и минимальной дат

df = (spark.read.option('header', True)
                .option('sep', ',')
                .option('inferSchema', True)
                .csv('covid-data.csv'))

max_date = df.agg(max("date")).collect()[0][0]
min_date = df.agg(min("date")).collect()[0][0]

print(f"Максимальная дата в датасете: {max_date}, а минимальная: {min_date}")

Максимальная дата в датасете: 2021-04-17, а минимальная: 2020-01-01


In [21]:
# Определим результат на 31/03/2020

(df.where((col('date') == '2020-03-31')
          & (~col('iso_code').like('%OWID_%')))
   .select('iso_code',
           col('location').alias('страна'),
           (round((col('total_cases') - col('total_deaths')) / col('population') * 100, 3)).alias('процент переболевших'))
   .orderBy(col('процент переболевших').desc())
   .limit(15)
   .show())

+--------+-----------+--------------------+
|iso_code|     страна|процент переболевших|
+--------+-----------+--------------------+
|     SMR| San Marino|               0.619|
|     AND|    Andorra|               0.471|
|     LUX| Luxembourg|               0.344|
|     ISL|    Iceland|               0.332|
|     CHE|Switzerland|               0.187|
|     ESP|      Spain|               0.187|
|     ITA|      Italy|               0.154|
|     MCO|     Monaco|                0.13|
|     AUT|    Austria|               0.112|
|     BEL|    Belgium|               0.104|
|     NOR|     Norway|               0.085|
|     DEU|    Germany|               0.085|
|     FRA|     France|               0.072|
|     PRT|   Portugal|               0.071|
|     NLD|Netherlands|               0.068|
+--------+-----------+--------------------+



In [22]:
# А также рейтинг стран по проценту переболевших на 31/03/2021

(df.where((col('date') == '2021-03-31')
          & (~col('iso_code').like('%OWID_%')))
   .select('iso_code',
           col('location').alias('страна'),
           (round((col('total_cases') - col('total_deaths')) / col('population') * 100, 3)).alias('процент переболевших'))
   .orderBy(col('процент переболевших').desc())
   .limit(15)
   .show())

+--------+-------------+--------------------+
|iso_code|       страна|процент переболевших|
+--------+-------------+--------------------+
|     AND|      Andorra|              15.395|
|     MNE|   Montenegro|              14.321|
|     CZE|      Czechia|              14.062|
|     SMR|   San Marino|               13.69|
|     SVN|     Slovenia|              10.176|
|     LUX|   Luxembourg|               9.728|
|     ISR|       Israel|               9.553|
|     USA|United States|               9.036|
|     SRB|       Serbia|               8.748|
|     BHR|      Bahrain|               8.458|
|     PAN|       Panama|               8.087|
|     EST|      Estonia|               7.955|
|     PRT|     Portugal|               7.893|
|     SWE|       Sweden|               7.836|
|     LTU|    Lithuania|               7.808|
+--------+-------------+--------------------+



# Top 10 стран с максимальным зафиксированным кол-вом новых случаев за последнюю неделю марта 2021 в отсортированном порядке по убыванию (в выходящем датасете необходимы колонки: число, страна, кол-во новых случаев)



In [25]:
(df.where((col('date').between('2021-03-28', '2021-03-31'))
          & (~col('iso_code').like('%OWID_%')))
   .orderBy(col('location'), col('new_cases').desc())
   .groupBy(col('location')).agg(first('new_cases').alias('new_cases'), first('date').alias('date'))
   .orderBy(col('new_cases').desc())
   .select(date_format('date', 'yyyy-MM-dd').alias('число'),
           col('location').alias('страна'),
           col('new_cases').alias('кол-во новых случаев'))
   .limit(10)
   .show())

+----------+-------------+--------------------+
|     число|       страна|кол-во новых случаев|
+----------+-------------+--------------------+
|2021-03-31|       Brazil|             90638.0|
|2021-03-31|        India|             72330.0|
|2021-03-29|United States|             69429.0|
|2021-03-31|       France|             59054.0|
|2021-03-31|       Turkey|             39302.0|
|2021-03-31|       Poland|             32891.0|
|2021-03-31|      Germany|             25014.0|
|2021-03-31|        Italy|             23887.0|
|2021-03-30|       Sweden|             16427.0|
|2021-03-31|    Argentina|             16056.0|
+----------+-------------+--------------------+



# Посчитайте изменение случаев относительно предыдущего дня в России за последнюю неделю марта 2021. (например: в россии вчера было 9150 , сегодня 8763, итог: -387) (в выходящем датасете необходимы колонки: число, кол-во новых случаев вчера, кол-во новых случаев сегодня, дельта)

In [33]:
(df.where((col('date').between('2021-03-27', '2021-03-31'))
          & (col('iso_code') == 'RUS'))
   .withColumn("кол-во новых случаев вчера",lag('new_cases').over(Window.orderBy("date")))
   .select(date_format('date', 'yyyy-MM-dd').alias('число'),
           col('new_cases').alias('кол-во новых случаев сегодня'),
           'кол-во новых случаев вчера',
           (col('new_cases') - col('кол-во новых случаев вчера')).alias('дельта'))
   .where(col('date') >= '2021-03-28')
   .show())

+----------+----------------------------+--------------------------+------+
|     число|кол-во новых случаев сегодня|кол-во новых случаев вчера|дельта|
+----------+----------------------------+--------------------------+------+
|2021-03-28|                      8979.0|                    8783.0| 196.0|
|2021-03-29|                      8589.0|                    8979.0|-390.0|
|2021-03-30|                      8162.0|                    8589.0|-427.0|
|2021-03-31|                      8156.0|                    8162.0|  -6.0|
+----------+----------------------------+--------------------------+------+

